In [30]:
import networkx as nx
import pandas as pd
import numpy as np
import scipy as sp
import scipy.sparse as sp

from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

In [12]:
df = pd.read_csv('data/yeast_wv', sep=' ', skiprows=[0], header=None)
df = df.set_index(df.columns[0])
df = df.sort_index()

In [13]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
0,,,,,,,,,,,,,,,,,,,,,
-,-0.358242,-0.251310,-0.020519,0.007021,0.191769,-0.515085,-0.254772,-0.056863,0.338159,0.173580,...,-0.255537,0.402859,0.276861,0.651923,-0.219958,-0.018561,0.258365,-0.099855,0.316638,-0.150632
AD-005,-0.239109,0.334299,0.077726,0.051477,-0.021562,-0.294491,-0.166697,0.208436,0.438678,0.052045,...,0.033514,-0.025393,0.413263,-0.235144,-0.700902,0.143463,0.084440,-0.394031,-0.052691,0.131935
AD-016,-0.495126,-0.179522,-0.231075,0.100351,0.158653,0.040835,-0.123357,0.055078,0.187738,0.064589,...,0.161642,0.062534,-0.019364,0.055374,-0.373510,-0.136465,0.433998,-0.045476,0.559419,-0.238186
AL049758.1,0.100712,-0.017481,-0.302953,0.009781,0.148234,0.334797,-0.201392,0.428964,0.131606,-0.210761,...,-0.327259,-0.301696,0.546253,-0.021461,-0.304658,0.207013,0.540333,-0.607864,0.262678,-0.114324
AT1G03060,0.170729,-0.277690,-0.614924,-0.031645,0.087958,-0.305242,-0.259962,0.090138,0.163315,0.003857,...,0.319380,0.126689,-0.213734,0.006822,0.117965,0.478203,0.096430,-0.527097,0.186286,-0.131853


In [24]:
G = nx.read_edgelist('data/yeast.edgelist')
adj = nx.adjacency_matrix(G, nodelist=sorted(G.nodes()))

In [16]:
sorted(G.nodes())

['-',
 'AD-005',
 'AD-016',
 'AL049758.1',
 'AT1G03060',
 'AT1G03190',
 'AT1G05055',
 'AT1G07370',
 'AT1G07880',
 'AT1G07910',
 'AT1G08370',
 'AT1G09020',
 'AT1G09330',
 'AT1G16890',
 'AT1G23260',
 'AT1G27310',
 'AT1G27970',
 'AT1G48600',
 'AT1G75950',
 'AT1G77140',
 'AT1G78870',
 'AT1G79650',
 'AT2G04030',
 'AT2G17200',
 'AT2G26590',
 'AT2G26650',
 'AT2G29570',
 'AT2G40650',
 'AT2G46800',
 'AT3G01090',
 'AT3G16720',
 'AT3G18000',
 'AT3G19210',
 'AT3G22200',
 'AT3G29160',
 'AT3G52560',
 'AT3G52590',
 'AT4G15900',
 'AT4G16360',
 'AT4G35800',
 'AT4G38630',
 'AT4G39850',
 'AT5G20850',
 'AT5G21170',
 'AT5G41700',
 'AT5G42190',
 'AT5G44740',
 'AT5G56580',
 'AT5G58580',
 'BM-015',
 'BOS_11463',
 'BOS_13459',
 'BOS_14437',
 'BOS_15646',
 'BOS_16579',
 'BOS_2150',
 'BOS_21644',
 'BOS_21792',
 'BOS_21808',
 'BOS_21879',
 'BOS_21927',
 'BOS_21938',
 'BOS_21942',
 'BOS_25889',
 'BOS_268',
 'BOS_2933',
 'BOS_2955',
 'BOS_5053',
 'BOS_5055',
 'BOS_7381',
 'C1_00930C_A',
 'C1_01570C_A',
 'C22:RP4-57

In [19]:
df.T.head()

,-,AD-005,AD-016,AL049758.1,AT1G03060,AT1G03190,AT1G05055,AT1G07370,AT1G07880,AT1G07910,...,tY(GUA)D,tY(GUA)F1,tY(GUA)F2,tY(GUA)J1,tY(GUA)J2,tY(GUA)M1,tY(GUA)M2,tY(GUA)O,tcag7.1178,tcag7.1300
1,-0.358242,-0.239109,-0.495126,0.100712,0.170729,-0.171528,-0.153754,-0.399418,-0.165045,-0.868675,...,-0.193367,-0.148972,-0.187666,-0.292264,-0.236823,-0.209958,-0.148576,-0.249109,-0.373755,-0.469923
2,-0.251310,0.334299,-0.179522,-0.017481,-0.277690,-0.310711,0.184084,0.201377,-0.088582,-0.217651,...,-0.075892,0.188809,-0.337989,-0.064790,-0.298546,-0.058807,-0.126903,-0.064573,0.077297,0.512941
3,-0.020519,0.077726,-0.231075,-0.302953,-0.614924,-0.412126,-0.126122,-0.186095,-0.246011,0.239841,...,-0.112438,-0.017260,-0.283403,-0.193466,-0.150377,-0.395206,-0.392373,-0.284234,-0.531538,-0.180259
4,0.007021,0.051477,0.100351,0.009781,-0.031645,0.222153,0.241257,-0.098676,0.054039,0.638674,...,-0.050001,-0.334032,-0.142003,-0.059666,0.525596,0.038261,-0.121760,0.153524,-0.044634,-0.249501
5,0.191769,-0.021562,0.158653,0.148234,0.087958,-0.222884,-0.204582,0.154032,0.124681,0.067799,...,-0.210795,0.186743,0.253187,-0.105121,-0.820546,-0.028616,0.060464,-0.047096,-0.261323,0.086728


In [23]:
# 内積を計算
sim = np.dot(np.array(df), np.array(df.T))
print(sim.shape)
sim

(6526, 6526)


array([[ 2.03937753,  0.9985998 ,  0.86354437, ...,  0.55709119,
         1.01705399,  0.87173565],
       [ 0.9985998 ,  4.18713925,  1.02630819, ...,  0.86588147,
        -0.29728468,  0.01920363],
       [ 0.86354437,  1.02630819,  1.68122996, ...,  0.77356822,
         0.20205074,  0.41397279],
       ...,
       [ 0.55709119,  0.86588147,  0.77356822, ...,  0.89593014,
         0.68551313,  0.53385063],
       [ 1.01705399, -0.29728468,  0.20205074, ...,  0.68551313,
         5.77679175,  1.94053482],
       [ 0.87173565,  0.01920363,  0.41397279, ...,  0.53385063,
         1.94053482,  2.96494882]])

In [33]:
def load_data(datapath):
    """
    edgelistデータを読み込み，　sp.sparse型の隣接行列を返す
    """
    g = nx.read_edgelist(datapath)
    adj = nx.adjacency_matrix(g, nodelist=sorted(g.nodes()))
    return adj.T


def weight_variable_glorot(input_dim, output_dim, name=""):
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = tf.random_uniform(
        [input_dim, output_dim], minval=-init_range,
        maxval=init_range, dtype=tf.float32)
    return tf.Variable(initial, name=name)


def dropout_sparse(x, keep_prob, num_nonzero_elems):
    noise_shape = [num_nonzero_elems]
    random_tensor = keep_prob
    random_tensor += tf.random_uniform(noise_shape)
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
    pre_out = tf.sparse_retain(x, dropout_mask)
    return pre_out * (1. / keep_prob)


def sparse_to_tuple(sparse_mx):
    """
    sp.sparse行列を，　値を持つcoords，　値の中身, 本来の行列の形のtupleに変換
    """
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape


def preprocess_graph(adj):
    """
    sp.sparse型の隣接行列を，　次数行列で正規化して，　coords, values, shape のtupleに変換
    """
    adj = sp.coo_matrix(adj)
    adj_ = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
    return sparse_to_tuple(adj_normalized)



def mask_test_edges(adj):
    # Function to build test set with 2% positive links (全体の1/50のリンク)
    # ここでは，　2%の正解に合わせて，　同じ数の不正解データも作っておく （精度を求める際に使用）
    
    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()

    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    num_test = int(np.floor(edges.shape[0] / 50.))
    num_val = int(np.floor(edges.shape[0] / 50.))
    
    # エッジのidをシャッフルして，　訓練用, 検証用と，テスト用のエッジに分ける
    all_edge_idx = [i for i in range(edges.shape[0])]
    np.random.shuffle(all_edge_idx)
    val_edge_idx = all_edge_idx[:num_val]
    test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
    test_edges = edges[test_edge_idx]
    val_edges = edges[val_edge_idx]
    train_edges = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)  # 配列edgesから，　テストと検証用のエッジを除く

    def ismember(a, b):
        rows_close = np.all((a - b[:, None]) == 0, axis=-1)
        return np.any(rows_close)

    test_edges_false = []
    while len(test_edges_false) < len(test_edges):
        n_rnd = len(test_edges) - len(test_edges_false) 
        rnd = np.random.randint(0, adj.shape[0], size=2 * n_rnd)  # 2*n_rnd 個のリスト（各要素は，　ランダムに選ばれたノードの番号）
        idxs_i = rnd[:n_rnd]  # 　半分を i と j それぞれに分ける                                      
        idxs_j = rnd[n_rnd:]
        for i in range(n_rnd):
            idx_i = idxs_i[i]
            idx_j = idxs_j[i]
            if idx_i == idx_j:
                continue
            if ismember([idx_i, idx_j], edges_all):
                continue
            if test_edges_false:  # リストが空でないなら，　True
                if ismember([idx_j, idx_i], np.array(test_edges_false)):
                    continue
                if ismember([idx_i, idx_j], np.array(test_edges_false)):
                    continue
            test_edges_false.append([idx_i, idx_j])  # 上記の条件を満たさない場合だけ，　test_edges_falseに加える

    val_edges_false = []
    while len(val_edges_false) < len(val_edges):
        n_rnd = len(val_edges) - len(val_edges_false)
        rnd = np.random.randint(0, adj.shape[0], size=2 * n_rnd)
        idxs_i = rnd[:n_rnd]                                        
        idxs_j = rnd[n_rnd:]
        for i in range(n_rnd):
            idx_i = idxs_i[i]
            idx_j = idxs_j[i]
            if idx_i == idx_j:
                continue
            if ismember([idx_i, idx_j], train_edges):
                continue
            if ismember([idx_j, idx_i], train_edges):
                continue
            if ismember([idx_i, idx_j], val_edges):
                continue
            if ismember([idx_j, idx_i], val_edges):
                continue
            if val_edges_false:
                if ismember([idx_j, idx_i], np.array(val_edges_false)):
                    continue
                if ismember([idx_i, idx_j], np.array(val_edges_false)):
                    continue
            val_edges_false.append([idx_i, idx_j])

    # Re-build adj matrix
    data = np.ones(train_edges.shape[0])
    adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
    adj_train = adj_train + adj_train.T

    return adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false


def get_roc_score(edges_pos, edges_neg):
    """
    評価用の，　正解エッジのリストと，　不正解エッジのリストを渡す
    """
    emb = sim

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Predict on test set of edges
    adj_rec = np.dot(emb, emb.T)  # embは(N, F)行列ゆえ，　左のように積を取ると，　(N, N)で(i, j)成分はノード i と j の類似度を表す
    
    # positive edge と negative edge の2つで別々に評価
    preds = []
    pos = []
    for e in edges_pos:
        preds.append(sigmoid(adj_rec[e[0], e[1]]))
        pos.append(adj_orig[e[0], e[1]])

    preds_neg = []
    neg = []
    for e in edges_neg:
        preds_neg.append(sigmoid(adj_rec[e[0], e[1]]))
        neg.append(adj_orig[e[0], e[1]])

    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)

    return roc_score, ap_score

In [35]:
# Store original adjacency matrix (without diagonal entries) for later
adj_orig = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
adj_orig.eliminate_zeros()

In [29]:
adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)

In [36]:
roc_score, ap_score = get_roc_score(test_edges, test_edges_false)
print('roc_score: %f'%roc_score)
print('ap_score: %f'%ap_score)

roc_score: 0.508295
ap_score: 0.504183
